In [ ]:
"""
Created on Thu Feb 08 10:42 2024

Look at some 2D variables in the Casimir runs

Author: @claraburgard
"""

In [1]:
import xarray as xr
import numpy as np

import matplotlib.pyplot as plt
import matplotlib as mpl
import cmocean

import cartopy.crs as ccrs
import cartopy.feature
from cartopy.util import add_cyclic_point
import seaborn as sns
import cartopy
import matplotlib.colors as colors
from matplotlib.colors import Normalize
from cartopy.feature import LAND

import gsw

from basal_melt_NEMO.constants import *
import basal_melt_NEMO.figure_functions as figf
import basal_melt_NEMO.useful_functions as uf

In [2]:
sns.set_context('paper')

In [3]:
%matplotlib qt5

QStandardPaths: error creating runtime directory '/run/user/2784' (Permission denied)


READ IN DATA

In [4]:
inputpath_closed='/data/cdelaver/n42tm21/'
inputpath_open='/data/cdelaver/n42openc/'
clara_path='/data/cburgard/CASIMIR_SIMU/interim/XR_PROCESSED/'
plot_path = '/data/cburgard/PLOTS/first_plots/'
mask_path = '/data/cburgard/TOOLS/'

In [5]:
ocean_masks = xr.open_dataset(mask_path + 'basin_masks_orca1_nemo4p2.nc')

In [6]:
file_open_mean10 = xr.open_mfdataset(inputpath_open + 'n42openc_00910101_01001231_1Y_grid_T.nc').mean('time_counter')
file_closed_mean10 = xr.open_mfdataset(inputpath_closed + 'n42tm21_00910101_01001231_1Y_grid_T.nc').mean('time_counter')

file_open_V_mean10 = xr.open_mfdataset(inputpath_open + 'n42openc_00910101_01001231_1Y_grid_V.nc').mean('time_counter')
file_closed_V_mean10 = xr.open_mfdataset(inputpath_closed + 'n42tm21_00910101_01001231_1Y_grid_V.nc').mean('time_counter')

In [7]:
file_open_mean10 = file_open_mean10.where(file_open_mean10['so'] > 0)
file_open_V_mean10 = file_open_V_mean10.where(file_open_mean10['so'].rename({'deptht':'depthv'}) > 0)

In [8]:
lon = file_open_mean10.nav_lon
lat = file_open_mean10.nav_lat

In [9]:
T_30W_closed = file_closed_mean10['thetao'].where((lon > -31.) & (lon < -29.)).mean('x')
T_30W_open = file_open_mean10['thetao'].where((lon > -31.) & (lon < -29.)).mean('x')

In [35]:
# checking if the line is right
#ref = file_closed_mean10['thetao'].where((lon > -31.) & (lon < -29.)).isel(deptht=0)
ref = ocean_masks['pacific']
plt.figure()

llon = lon
llat = lat

proj = ccrs.PlateCarree(central_longitude=0)
wrap_ref, wrap_lon = ref, lon #add_cyclic_point(ref.values,coord=lon,axis=1)

theta = np.linspace(0, 2*np.pi, 100)
center, radius = [0.5, 0.5], 0.5
verts = np.vstack([np.sin(theta), np.cos(theta)]).T
circle = mpl.path.Path(verts * radius + center)   

ax1 = plt.subplot(1, 1, 1, projection=proj)
abso0 = ax1.pcolormesh(wrap_lon,lat,wrap_ref,transform=ccrs.PlateCarree(),rasterized=True)
ax1.coastlines(resolution='110m', linewidth=0.5)
ax1.set_extent([-180, 180, -90, 90], crs=ccrs.PlateCarree())
ax1.set_boundary(circle, transform=ax1.transAxes)

In [10]:
def compute_density(da):
    depth = da['e3t'].cumsum('deptht')
    S = da['so']
    T = da['thetao']
    return gsw.rho(S,T,depth)

In [11]:
def compute_sigma2(da):
    S = da['so']
    T = da['thetao']
    return gsw.density.sigma2(S, T)

In [28]:
def transect_along_ocean(var, lon, lon_min, lon_max, lat, lat_max, cpalette, varname):

    if lat_max == 90:
        plot_var_closed = file_closed_mean10[var].where((lon > lon_min) & (lon < lon_max) & (lat <= lat_max)).mean('x')
        plot_var_open = file_open_mean10[var].where((lon > lon_min) & (lon < lon_max) & (lat <= lat_max)).mean('x')
    else:
        plot_var_closed = file_closed_mean10[var].where((lon > lon_min) & (lon < lon_max) & (lat <= lat_max), drop=True).mean('x')
        plot_var_open = file_open_mean10[var].where((lon > lon_min) & (lon < lon_max) & (lat <= lat_max), drop=True).mean('x')
    
    f, ax = plt.subplots(3, 1, sharex=True, sharey=True, figsize=(8.25,8.25*3))

    abs1 = ax[0].contourf(plot_var_closed.y,-1*plot_var_closed.deptht, plot_var_closed, cmap=cpalette)
    f.colorbar(abs1, ax=ax[0], orientation='vertical')
    ax[0].set_title(varname)
    
    abs2 = ax[1].contourf(plot_var_open.y,-1*plot_var_open.deptht, plot_var_open, cmap=cpalette)
    f.colorbar(abs2, ax=ax[1], orientation='vertical')
    
    ax_limit = (np.abs(plot_var_open - plot_var_closed)).max().values
    abs3 = ax[2].contourf(plot_var_open.y,-1*plot_var_open.deptht, (plot_var_open - plot_var_closed), cmap=mpl.cm.coolwarm, vmin=-ax_limit, vmax=ax_limit)
    f.colorbar(abs3, ax=ax[2], orientation='vertical')

In [13]:
def streamfunction_along_ocean(vo_open, vo_closed, ocean, cpalette):
    
    vmass_sum_open = vo_open.where(np.isfinite(ocean_masks[ocean])).sum('x') / 10**6
    vmass_sum_closed = vo_closed.where(np.isfinite(ocean_masks[ocean])).sum('x') / 10**6

    vmass_sum_open = vmass_sum_open.assign_coords({'depthv': vmass_sum_open.depthv[::-1]})
    vmass_sum_closed = vmass_sum_closed.assign_coords({'depthv': vmass_sum_closed.depthv[::-1]})

    plot_var_open = vmass_sum_open.cumsum('depthv', skipna=True)
    plot_var_closed = vmass_sum_closed.cumsum('depthv', skipna=True)
    
    f, ax = plt.subplots(3, 1, sharex=True, sharey=True, figsize=(8.25,8.25*3))

    ax_limit0 = 20 #np.array([np.abs(plot_var_open),np.abs(plot_var_closed)]).max()
    
    abs1 = ax[0].contourf(plot_var_closed.y,-1*plot_var_closed.depthv, plot_var_closed, cmap=cpalette, vmin=-ax_limit0, vmax=ax_limit0)
    f.colorbar(abs1, ax=ax[0], orientation='vertical')
    ax[0].set_title('Streamfunction '+ocean)

    abs2 = ax[1].contourf(plot_var_open.y,-1*plot_var_open.depthv, plot_var_open, cmap=cpalette, vmin=-ax_limit0, vmax=ax_limit0)
    f.colorbar(abs2, ax=ax[1], orientation='vertical')
    
    ax_limit = (np.abs(plot_var_open - plot_var_closed)).max().values
    abs3 = ax[2].contourf(plot_var_open.y,-1*plot_var_open.depthv, (plot_var_open - plot_var_closed), cmap=mpl.cm.coolwarm, vmin=-ax_limit, vmax=ax_limit)
    f.colorbar(abs3, ax=ax[2], orientation='vertical')

In [13]:
### ATLANTIC

In [26]:
f_temp = transect_along_ocean('thetao', lon, -31., -29., lat, 90, cmocean.cm.thermal,'Conservative Temperature [°C]')

In [15]:
f_sal = transect_along_ocean('so', lon, -31., -29., lat, 90,cmocean.cm.haline,'Absolute Salinity [g/kg]')

In [33]:
file_closed_mean10['rho'] = compute_density(file_closed_mean10).load()
file_open_mean10['rho'] = compute_density(file_open_mean10).load()

In [17]:
f_rho = transect_along_ocean('rho', lon, -31., -29., lat, 90, cmocean.cm.dense,'In-situ density [kg/m3]')

In [34]:
file_closed_mean10['sigma2'] = compute_sigma2(file_closed_mean10).load()
file_open_mean10['sigma2'] = compute_sigma2(file_open_mean10).load()

In [20]:
f_sigma2 = transect_along_ocean('sigma2', lon, -31., -29., lat, 90,cmocean.cm.dense,'Potential density anomaly with reference pressure of 2000 dbar [kg/m3]')

In [ ]:
### PACIFIC

In [22]:
f_temp = transect_along_ocean('thetao', lon, -171, -169.,lat, 90,cmocean.cm.thermal,'Conservative Temperature [°C]')

In [23]:
f_sal = transect_along_ocean('so', lon,  -171, -169.,lat, 90,cmocean.cm.haline,'Absolute Salinity [g/kg]')

In [24]:
f_rho = transect_along_ocean('rho', lon, -171, -169.,lat, 90,cmocean.cm.dense,'In-situ density [kg/m3]')

In [25]:
f_sigma2 = transect_along_ocean('sigma2', lon, -171, -169.,lat, 90,cmocean.cm.dense,'Potential density anomaly with reference pressure of 2000 dbar [kg/m3]')

In [ ]:
### WEDDELL SEA

In [29]:
f_temp = transect_along_ocean('thetao', lon, -51, -49,lat, -50,cmocean.cm.thermal,'Conservative Temperature [°C] Weddell Sea')

In [31]:
f_sal = transect_along_ocean('so', lon,  -51, -49,lat,  -50,cmocean.cm.haline,'Absolute Salinity [g/kg] Weddell Sea')

In [36]:
f_sigma2 = transect_along_ocean('sigma2', lon, -51, -49,lat, -50,cmocean.cm.dense,'Potential density anomaly with reference pressure of 2000 dbar [kg/m3] Weddell Sea')

In [ ]:
### ROSS SEA

In [37]:
f_temp = transect_along_ocean('thetao', lon, -180, -179,lat, -50,cmocean.cm.thermal,'Conservative Temperature [°C] Ross Sea')

In [38]:
f_sal = transect_along_ocean('so', lon,  -180, -179,lat,  -50,cmocean.cm.haline,'Absolute Salinity [g/kg] Ross Sea')

In [39]:
f_sigma2 = transect_along_ocean('sigma2', lon, -180, -179,lat, -50,cmocean.cm.dense,'Potential density anomaly with reference pressure of 2000 dbar [kg/m3] Ross Sea')

In [ ]:
mask_Ross = ((lon <= -120) | (lon >= 150)) & (lat >= -87.) & (lat <= -65.)

In [ ]:
# STREAM FUNCTION

In [127]:
streamfunction_along_ocean(file_open_V_mean10['vocetr_eff'],file_closed_V_mean10['vocetr_eff'],'atlantic',mpl.cm.coolwarm)

In [14]:
v_atlsum_open = file_open_V_mean10['vocetr_eff'].where(np.isfinite(ocean_masks['atlantic'])).sum('x') / 10**6
v_atlsum_closed = file_closed_V_mean10['vocetr_eff'].where(np.isfinite(ocean_masks['atlantic'])).sum('x') / 10**6

In [15]:
v_atlsum_open_rev = v_atlsum_open.assign_coords({'depthv': v_atlsum_open.depthv[::-1]})
v_atlsum_closed_rev = v_atlsum_closed.assign_coords({'depthv': v_atlsum_closed.depthv[::-1]})

In [16]:
streamf_atl_open = v_atlsum_open.cumsum('depthv')
streamf_atl_closed = v_atlsum_closed.cumsum('depthv')

In [17]:
streamf_atl_open.plot(vmax=20)

libGL error: No matching fbConfigs or visuals found
libGL error: failed to load driver: swrast


In [22]:
streamf_atl_open_rev = v_atlsum_open_rev.cumsum('depthv')
streamf_atl_closed_rev = v_atlsum_closed_rev.cumsum('depthv').assign_coords({'depthv': -1*v_atlsum_closed_rev.depthv})


In [23]:
plt.figure()
streamf_atl_closed_rev.plot(vmax=20)

In [86]:
streamf_atl_open.plot()

In [64]:
file_open_V_mean10['vocetr_eff'].where(np.isfinite(ocean_masks['atlantic'])).isel(depthv=0).plot()

In [110]:
v_atlsum_open.plot()

In [117]:
v_atlsum_open.cumsum('depthv').plot()

In [ ]:
plt.figure()
v_atlsum_open_rev.cumsum('depthv').plot()